In [ ]:
tpu_use = False
if tpu_use == True:
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#     !python3 pytorch-xla-env-setup.py --version 1.9 --apt-packages libomp5 libopenblas-dev   
#     !python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev 

# Give up for using TPU 
```
!python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
```
```
Numpy not Available
```
-> !python3 pytorch-xla-env-setup.py --version 1.7 or more
is recommended


```
!python3 pytorch-xla-env-setup.py --version 1.9 --apt-packages libomp5 libopenblas-dev
```
```
RuntimeError: tensorflow/compiler/xla/xla_client/xrt_computation_client.cc:421 : Check failed: session->session()->Run(session_work->feed_inputs, session_work->outputs_handles, &outputs) == ::tensorflow::Status::OK() (Aborted: Session bae0b7f7935a353c is not found. vs. OK)
*** Begin stack trace ***
	tensorflow::CurrentStackTrace()
	
	xla::util::MultiWait::Complete(std::function<void ()> const&)
```
-> !python3 pytorch-xla-env-setup.py --version nightly
is recommended

Loop

In [ ]:
import sys
import glob
import os
import cv2
from pathlib import Path
import random
import time
from tqdm.notebook import tqdm
import itertools
import shutil

import numpy as np 
import pandas as pd 

import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import random_split, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
if tpu_use == True:
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.xla_multiprocessing as xmp
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.utils.serialization as xser
    device = xm.xla_device()
else:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Load Data

In [ ]:
def get_data_as_nparray(dir):
    data = []
    for i, image_path in enumerate(glob.glob(os.path.join(dir,'*.jpg'))):
        if i > 5:
            continue
        img = cv2.imread(image_path) # (height,width,channels)
        img_expanded = np.expand_dims(img,axis=0) # (1,height,width,channels)
        data.append(img_expanded)
    # (n_samples,height,width,channels)
    data_np = np.concatenate(data,axis=0)
    return data_np

def show_imgs(imd_data,height,width):
    %matplotlib inline
    # Show multi img
    plt.figure(figsize=(12,12))
    for i,d in enumerate(imd_data):
        if i >= width*height:         
            continue
        plt.subplot(height,width,i+1)
        plt.imshow(ｄ)

In [ ]:
style_dir = '/kaggle/input/gan-getting-started/monet_jpg/'
style_data = get_data_as_nparray(style_dir)
show_imgs(style_data,1,3)
print(style_data.shape)

In [ ]:
photo_dir = '/kaggle/input/gan-getting-started/photo_jpg/'
photo_data = get_data_as_nparray(photo_dir)
show_imgs(photo_data,1,3)


## random,torch　ensuring reproducibility
[reference](https://www.google.com/search?q=++++torch.backends.cudnn.deterministic+qiita&sxsrf=AOaemvKxBaC4koa_BLVExQIaGCbCwYaxHg%3A1630678601472&ei=SS4yYdejHI7_0ASTyIuQAg&oq=++++torch.backends.cudnn.deterministic+qiita&gs_lcp=Cgdnd3Mtd2l6EAM6BwgjELADECdKBAhBGAFQnBZYnBZgyx5oAXAAeACAAT2IAT2SAQExmAEAoAECoAEByAEBwAEB&sclient=gws-wiz&ved=0ahUKEwiX7KGW_-LyAhWOP5QKHRPkAiIQ4dUDCA4&uact=5)

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)   # pythonのハッシュベースの操作の再現性を担保
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False      # False:再現性確保 True：速度確保
    torch.backends.cudnn.deterministic = True   # PyTorchの操作の中には非決定的なものがあります。それらを決定的なものにします。

In [ ]:
seed = 95
set_seed(seed)

# Create Dataset with Shuffled Pair
[reference](https://qiita.com/kumonk/items/0f3cad018cc9aec67a63)

In [ ]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, style_dir, photo_dir, imageSize=(256,256), transform=None, normalize=True, diffPairAugmentRate=5):
        if normalize:
            self.transform = transforms.Compose([
                transforms.Resize(imageSize), 
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(imageSize), 
                transforms.ToTensor(), 
            ])
        # Dataのパスリスト
        self.style_paths = [str(p) for p in Path(style_dir).glob("*.jpg")]*diffPairAugmentRate
        self.photo_paths = [str(p) for p in Path(photo_dir).glob("*.jpg")]*diffPairAugmentRate
        # Shuffle Pairs of input and grandtruth data
        random.shuffle(self.style_paths)
        random.shuffle(self.photo_paths)
        self.data_num = min(len(self.style_paths),len(self.photo_paths)) # ここが__len__の返り値になる


    def __getitem__(self, idx):
        s = self.style_paths[idx]
        style_img = PIL.Image.open(s)
        p = self.photo_paths[idx]
        photo_img = PIL.Image.open(p)

        if self.transform:
            out_style = self.transform(style_img)
            out_photo = self.transform(photo_img)            
        
        # ForLabel
        # out_label = p.split("\\")
        # out_label = self.class_to_idx[out_label[3]]

        return out_photo, out_style,
        
    def __len__(self):
        return self.data_num

In [ ]:
# dataset = MyDataset(style_dir, photo_dir, normalize=False)
diffPairAugmentRate=10
dataset = MyDataset(style_dir, photo_dir, normalize=True,diffPairAugmentRate=diffPairAugmentRate)
train_size = int(len(dataset) * 0.95) 
val_size = len(dataset) - train_size 
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
# !python -m pip install numpy==1.18.3
# import numpy as np
# style_img, photo_img = next(iter(dataset))

# print(len(style_img))
# print(style_img.size())
# show_imgs([style_img.permute(1, 2, 0),photo_img.permute(1, 2, 0)],1,2)

In [ ]:
def unnorm(img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    for t, m, s in zip(img, mean, std):
        t.mul_(s).add_(s)  
    return img

# show_imgs([unnorm(style_img).permute(1, 2, 0),unnorm(photo_img).permute(1, 2, 0)],1,2)

## DataLoader ensuring reproducibility
[reference](https://www.google.com/search?q=++++torch.backends.cudnn.deterministic+qiita&sxsrf=AOaemvKxBaC4koa_BLVExQIaGCbCwYaxHg%3A1630678601472&ei=SS4yYdejHI7_0ASTyIuQAg&oq=++++torch.backends.cudnn.deterministic+qiita&gs_lcp=Cgdnd3Mtd2l6EAM6BwgjELADECdKBAhBGAFQnBZYnBZgyx5oAXAAeACAAT2IAT2SAQExmAEAoAECoAEByAEBwAEB&sclient=gws-wiz&ved=0ahUKEwiX7KGW_-LyAhWOP5QKHRPkAiIQ4dUDCA4&uact=5)
## DataLoader speedy
```
num_workers=os.cpu_count()
pin_memory=True
```
[reference](https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587#12-pin_memory)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

## Checkpoint Load and Save

In [ ]:
def load_checkpoint(ckpt_path, map_location=None):
    ckpt = torch.load(ckpt_path, map_location=map_location)
    print(' [*] Loading checkpoint from %s succeed!' % ckpt_path)
    return ckpt
def save_checkpoint(state, save_path):
    if tpu_use==True:
        xm.save(state, save_path)
    else:
        torch.save(state, save_path)

# Model

[ReflectionPad2d](https://teratail.com/questions/254795)

In [ ]:
def Upsample(in_ch, out_ch, use_dropout=True, dropout_ratio=0.1):
    if use_dropout:
        return nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(out_ch),
            nn.Dropout(dropout_ratio),
            nn.GELU()
        )
    else:
        return nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(out_ch),
            nn.GELU()
        )
    
def Convlayer(in_ch, out_ch, kernel_size=3, stride=1, use_leaky=True, use_inst_norm=True, use_pad=True):
    if use_pad:
        conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, 1, bias=True)
    else:
        conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, 0, bias=True)

    if use_leaky:
        actv = nn.LeakyReLU(negative_slope=0.2, inplace=True)
    else:
        actv = nn.GELU()

    if use_inst_norm:
        norm = nn.InstanceNorm2d(out_ch)
    else:
        norm = nn.BatchNorm2d(out_ch)

    return nn.Sequential(
        conv,
        norm,
        actv
    )

class Resblock(nn.Module):
    def __init__(self, in_features, use_dropout=True, dropout_ratio=0.1):
        super().__init__()
        layers = list()
        layers.append(nn.ReflectionPad2d(1))
        layers.append(Convlayer(in_features, in_features, 3, 1, use_leaky=True, use_inst_norm=True, use_pad=False))
        layers.append(nn.Dropout(dropout_ratio))
        layers.append(nn.ReflectionPad2d(1))
        layers.append(nn.Conv2d(in_features, in_features, 3, 1, padding=0, bias=True))
        layers.append(nn.InstanceNorm2d(in_features))
        self.res = nn.Sequential(*layers)

    def forward(self, x):
        return x + self.res(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_ch, out_ch, num_res_blocks=6):
        super().__init__()
        model = list()
        model.append(nn.ReflectionPad2d(3))
        model.append(Convlayer(in_ch, 64, kernel_size=7, stride=1, use_leaky=True, use_inst_norm=True, use_pad=False))
        model.append(Convlayer(64, 128, kernel_size=3, stride=2, use_leaky=True, use_inst_norm=True, use_pad=True))
        model.append(Convlayer(128, 256, kernel_size=3, stride=2, use_leaky=True, use_inst_norm=True, use_pad=True))
        for _ in range(num_res_blocks):
            model.append(Resblock(256))
        model.append(Upsample(256, 128))
        model.append(Upsample(128, 64))
        model.append(nn.ReflectionPad2d(3))
        model.append(nn.Conv2d(64, out_ch, kernel_size=7, stride=1, padding=0))
        model.append(nn.Tanh())

        self.gen = nn.Sequential(*model)

    def forward(self, x):
        return self.gen(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_ch, num_layers=4):
        super().__init__()
        model = list()
        model.append(nn.Conv2d(in_ch, 64, kernel_size=4, stride=2, padding=1))
        model.append(nn.LeakyReLU(negative_slope=0.2, inplace=True))
        for i in range(1, num_layers):
            in_chs = 64 * 2**(i-1)
            out_chs = in_chs * 2
            if i == num_layers -1:
                model.append(Convlayer(in_chs, out_chs, kernel_size=4, stride=1))
            else:
                model.append(Convlayer(in_chs, out_chs, kernel_size=4, stride=2))
        model.append(nn.Conv2d(out_chs, 1, kernel_size=4, stride=1, padding=1))
#         model.append(nn.Sigmoid())
        self.disc = nn.Sequential(*model)

    def forward(self, x):
        return self.disc(x)

In [ ]:
# def init_weights(net, init_type='normal', gain=0.02):
#     def init_func(m):
#         classname = m.__class__.__name__
#         if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
#             init.normal_(m.weight.data, 0.0, gain)
#             if hasattr(m, 'bias') and m.bias is not None:
#                 init.constant_(m.bias.data, 0.0)
#         elif classname.find('BatchNorm2d') != -1:
#             init.normal_(m.weight.data, 1.0, gain)
#             init.constant_(m.bias.data, 0.0)
#     net.apply(init_func)
def init_weights(m, init_type='normal', gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, gain)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:
            init.normal_(m.weight.data, 1.0, gain)
            init.constant_(m.bias.data, 0.0)

In [ ]:
def update_req_grad(models, requires_grad=True):
    for model in models:
        for param in model.parameters():
            param.requires_grad = requires_grad
            
# https://arxiv.org/pdf/1612.07828.pdf
# Save 50 generated fake imgs and sample through them
# to feed discriminators to avoid large oscillations 
# from iterations to iterations.
class sample_fake(object):
    def __init__(self, max_imgs=50):
        self.max_imgs = max_imgs
        self.cur_img = 0
        self.imgs = list()

    def __call__(self, imgs):
        ret = list()
        for img in imgs:
            if self.cur_img < self.max_imgs:
                self.imgs.append(img)
                ret.append(img)
                self.cur_img += 1
            else:
                if np.random.ranf() > 0.5:
                    idx = np.random.randint(0, self.max_imgs)
                    ret.append(self.imgs[idx])
                    self.imgs[idx] = img
                else:
                    ret.append(img)
        return ret
    
class lr_sched():
    def __init__(self, decay_epochs=20, total_epochs=40):
        self.decay_epochs = decay_epochs
        self.total_epochs = total_epochs

    def step(self, epoch_num):
        if epoch_num <= self.decay_epochs:
            return 1.0
        else:
            fract = (epoch_num - self.decay_epochs)  / (self.total_epochs - self.decay_epochs)
            return 1.0 - fract

class AvgStats(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses =[]
        self.its = []
        
    def append(self, loss, it):
        self.losses.append(loss)
        self.its.append(it)

# GAN

In [ ]:
class CycleGAN(object):
    def __init__(self, in_ch, out_ch, epochs, device, start_lr=5e-4, lmbda=10, idt_coef=0.5, decay_epoch=0):
        self.epochs = epochs
        self.decay_epoch = decay_epoch if decay_epoch > 0 else int(self.epochs/2)
        self.lmbda = lmbda
        self.idt_coef = idt_coef
        self.device = device
        if tpu_use == True:
            self.gen_s2p = xmp.MpModelWrapper(Generator(in_ch, out_ch))
            self.gen_p2s = xmp.MpModelWrapper(Generator(in_ch, out_ch))
            self.desc_s = xmp.MpModelWrapper(Discriminator(in_ch))
            self.desc_p = xmp.MpModelWrapper(Discriminator(in_ch))
        else:   
            self.gen_s2p = Generator(in_ch, out_ch)
            self.gen_p2s = Generator(in_ch, out_ch)
            self.desc_s = Discriminator(in_ch)
            self.desc_p = Discriminator(in_ch)
        self.init_models()
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()
#         self.adam_gen = torch.optim.Adam(itertools.chain(self.gen_s2p.parameters(), self.gen_p2s.parameters()),
#                                          lr= start_lr, betas=(0.5, 0.999))
#         self.adam_desc = torch.optim.Adam(itertools.chain(self.desc_s.parameters(), self.desc_p.parameters()),
#                                           lr=start_lr, betas=(0.5, 0.999))
        self.adam_gen = torch.optim.AdamW(itertools.chain(self.gen_s2p.parameters(), self.gen_p2s.parameters()),
                                         lr= start_lr)
        self.adam_desc = torch.optim.AdamW(itertools.chain(self.desc_s.parameters(), self.desc_p.parameters()),
                                          lr=start_lr)
        self.sample_style = sample_fake()
        self.sample_photo = sample_fake()
        gen_lr = lr_sched(self.decay_epoch, self.epochs)
        desc_lr = lr_sched(self.decay_epoch, self.epochs)
        self.gen_lr_sched = torch.optim.lr_scheduler.LambdaLR(self.adam_gen, gen_lr.step)
        self.desc_lr_sched = torch.optim.lr_scheduler.LambdaLR(self.adam_desc, desc_lr.step)
        self.gen_stats = AvgStats()
        self.desc_stats = AvgStats()
        self.val_gen_stats =  AvgStats()
        self.val_desc_stats =  AvgStats()
        
    def init_models(self):
        init_weights(self.gen_s2p)
        init_weights(self.gen_p2s)
        init_weights(self.desc_s)
        init_weights(self.desc_p)
        self.gen_s2p = self.gen_s2p.to(self.device)
        self.gen_p2s = self.gen_p2s.to(self.device)
        self.desc_s = self.desc_s.to(self.device)
        self.desc_p = self.desc_p.to(self.device)
    
    def make_dataloader(self,train_dataset, val_dataset):
        if tpu_use==True:
            train_sampler = DistributedSampler(
                train_dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=True
                )
            val_sampler = DistributedSampler(
                val_dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=True
                )
            train_sampler.set_epoch(self.epochs)
            val_sampler.set_epoch(self.epochs)

#             g = torch.Generator()
#             g.manual_seed(seed)
            img_train = DataLoader(train_dataset,
                                    batch_size=1,
                                    sampler=train_sampler, 
                                    num_workers=os.cpu_count(),
                                    worker_init_fn=seed_worker,
                                    generator=g,
                                    pin_memory=True)
            img_val = DataLoader(val_dataset,
                                    batch_size=1,
                                    sampler=val_sampler, 
                                    num_workers=os.cpu_count(),
                                    worker_init_fn=seed_worker,
                                    generator=g,
                                    pin_memory=True)
            img_train = pl.ParallelLoader(img_train, [device]).per_device_loader(device)
            img_val = pl.ParallelLoader(img_val, [device]).per_device_loader(device)
        
        
        else:  
            img_train = DataLoader(train_dataset,
                                    batch_size=1,
                                    num_workers=os.cpu_count(),
                                    worker_init_fn=seed_worker,
#                                     generator=g,
                                    pin_memory=True)
            img_val = DataLoader(val_dataset,
                                    batch_size=1,
                                    num_workers=os.cpu_count(),
                                    worker_init_fn=seed_worker,
#                                     generator=g,
                                    pin_memory=True) 
            return img_train, img_val
    
    def run_gen(self,photo_img, style_img):
        # Forward pass through generator
        fake_photo = self.gen_s2p(style_img)
        fake_style = self.gen_p2s(photo_img)

        cycl_style = self.gen_p2s(fake_photo)
        cycl_photo = self.gen_s2p(fake_style)

        id_style = self.gen_p2s(style_img)
        id_photo = self.gen_s2p(photo_img)

        # generator losses - identity, Adversarial, cycle consistency
        idt_loss_style = self.l1_loss(id_style, style_img) * self.lmbda * self.idt_coef
        idt_loss_photo = self.l1_loss(id_photo, photo_img) * self.lmbda * self.idt_coef

        cycle_loss_style = self.l1_loss(cycl_style, style_img) * self.lmbda
        cycle_loss_photo = self.l1_loss(cycl_photo, photo_img) * self.lmbda

        style_desc = self.desc_s(fake_style)
        photo_desc = self.desc_p(fake_photo)

        real = torch.ones(style_desc.size()).to(self.device)

        adv_loss_style = self.mse_loss(style_desc, real)
        adv_loss_photo = self.mse_loss(photo_desc, real)

        # total generator loss
        total_gen_loss = cycle_loss_style + adv_loss_style\
                      + cycle_loss_photo + adv_loss_photo\
                      + idt_loss_style + idt_loss_photo
        return total_gen_loss, fake_photo, fake_style
    
    def run_desc(self, photo_img, style_img, fake_style, fake_photo):
        fake_style = torch.tensor(fake_style).to(self.device)
        fake_photo = torch.tensor(fake_photo).to(self.device)

        style_desc_real = self.desc_s(style_img)
        style_desc_fake = self.desc_s(fake_style)
        photo_desc_real = self.desc_p(photo_img)
        photo_desc_fake = self.desc_p(fake_photo)

        real = torch.ones(style_desc_real.size()).to(self.device)
        fake = torch.zeros(style_desc_fake.size()).to(self.device)

        # Descriminator losses
        # --------------------
        style_desc_real_loss = self.mse_loss(style_desc_real, real)
        style_desc_fake_loss = self.mse_loss(style_desc_fake, fake)
        photo_desc_real_loss = self.mse_loss(photo_desc_real, real)
        photo_desc_fake_loss = self.mse_loss(photo_desc_fake, fake)

        style_desc_loss = (style_desc_real_loss + style_desc_fake_loss) / 2
        photo_desc_loss = (photo_desc_real_loss + photo_desc_fake_loss) / 2
        total_desc_loss = style_desc_loss + photo_desc_loss
        
        return total_desc_loss, style_desc_loss, photo_desc_loss
    
    def train(self, train_dataset,val_dataset,diffPairAugmentRate):
        best_val=99999
        img_train, img_val = self.make_dataloader(train_dataset, val_dataset)
        for epoch in range(self.epochs):
            
            start_time = time.time()
            avg_gen_loss = 0.0
            avg_desc_loss = 0.0
            val_avg_gen_loss = 0.0
            val_avg_desc_loss = 0.0
            
            # For cut augmented dataset
            original_len_val = img_val.__len__() / diffPairAugmentRate
            idx_max_val = original_len_val  * (diffPairAugmentRate-1) - 1 
                
            original_len_train = img_train.__len__() / diffPairAugmentRate
            idx_max_train = original_len_train * (diffPairAugmentRate-1) - 1 
            
            idx_start_val = random.randint(0,idx_max_val)
            idx_start_train = random.randint(0,idx_max_train)
            
            # Validation
            self.gen_s2p.eval()
            self.gen_p2s.eval()
            self.desc_s.eval()
            self.desc_p.eval()

            for i, (photo_real, style_real) in enumerate(img_val):
                if i < idx_start_val:
                    continue
                elif i >= idx_start_val + original_len_val:
                    continue
                    
                with torch.no_grad():
                    # Generator
                    photo_img, style_img = photo_real.to(device), style_real.to(device)
                    total_gen_loss, fake_photo, fake_style= self.run_gen(photo_img, style_img)
                    # Descriminator
                    fake_style = self.sample_style([fake_style.cpu().data.numpy()])[0]
                    fake_photo = self.sample_photo([fake_photo.cpu().data.numpy()])[0]
                    total_desc_loss, style_desc_loss, photo_desc_loss = self.run_desc(photo_img, style_img, fake_style, fake_photo)                 
                    
                    val_avg_gen_loss += total_gen_loss.item()
                    val_avg_desc_loss += total_desc_loss.item()
                    
            # Train
            t = tqdm(img_train, leave=False, total=img_train.__len__())
            self.gen_s2p.train()
            self.gen_p2s.train()
            self.desc_s.train()
            self.desc_p.train()
            for i, (photo_real, style_real) in enumerate(t):
                if i < idx_start_train:
                    continue
                elif i >= idx_start_train + original_len_train:
                    continue
                photo_img, style_img = photo_real.to(device), style_real.to(device)
                
                # Generator
                update_req_grad([self.desc_s, self.desc_p], False)
                self.adam_gen.zero_grad()
                total_gen_loss, fake_photo, fake_style= self.run_gen(photo_img, style_img)
                avg_gen_loss += total_gen_loss.item()
                total_gen_loss.backward()
                self.adam_gen.step()

                # Descriminator
                update_req_grad([self.desc_s, self.desc_p], True)
                self.adam_desc.zero_grad()
                fake_style = self.sample_style([fake_style.cpu().data.numpy()])[0]
                fake_photo = self.sample_photo([fake_photo.cpu().data.numpy()])[0]
                total_desc_loss, style_desc_loss, photo_desc_loss = self.run_desc(photo_img, style_img, fake_style, fake_photo)
                avg_desc_loss += total_desc_loss.item()

                # Backward
                style_desc_loss.backward()
                photo_desc_loss.backward()
                self.adam_desc.step()
                
                t.set_postfix(gen_loss=total_gen_loss.item(), desc_loss=total_desc_loss.item())
            
            val_avg_gen_loss /= img_val.__len__()
#             val_avg_desc_loss /= img_val.__len__()
            val_avg_desc_loss /= img_val.__len__()*0.1 # for easy-look graph
           
            avg_gen_loss /= img_train.__len__()
#             avg_desc_loss /= img_train.__len__()
            avg_desc_loss /= img_train.__len__()*0.1 # for easy-look graph
            time_req = time.time() - start_time

            val_loss = val_avg_gen_loss+val_avg_desc_loss
            if best_val >= val_loss:
    
                save_dict = {
                    'epoch': epoch+1,
                    'gen_s2p': gan.gen_s2p.state_dict(),
                    'gen_p2s': gan.gen_p2s.state_dict(),
                    'desc_s': gan.desc_s.state_dict(),
                    'desc_p': gan.desc_p.state_dict(),
                    'optimizer_gen': gan.adam_gen.state_dict(),
                    'optimizer_desc': gan.adam_desc.state_dict()
                }
                save_checkpoint(save_dict, 'current.ckpt')

            self.val_gen_stats.append(val_avg_gen_loss, time_req)
            self.val_desc_stats.append(val_avg_desc_loss, time_req)
            self.gen_stats.append(avg_gen_loss, time_req)
            self.desc_stats.append(avg_desc_loss, time_req)
            
            if tpu_use == True:    
                xm.master_print("Epoch: (%d) | Gen ValLoss:%f | Disc ValLoss:%f " % 
                        (epoch+0, val_avg_gen_loss,val_avg_desc_loss))
                xm.master_print("Epoch: (%d) | Gen TraLoss:%f | Disc TraLoss:%f " % 
                        (epoch+1, avg_gen_loss, avg_desc_loss))
            else:
                print("Epoch: (%d) | Gen ValLoss:%f | Disc ValLoss:%f " % 
                        (epoch+0, val_avg_gen_loss,val_avg_desc_loss))
                print("Epoch: (%d) | Gen TraLoss:%f | Disc TraLoss:%f " % 
                        (epoch+1, avg_gen_loss, avg_desc_loss))
            self.gen_lr_sched.step()
            self.desc_lr_sched.step()

In [ ]:
gan = CycleGAN(3, 3, 100, device)
if tpu_use == True:
    xmp.spawn(gan.train(train_dataset,val_dataset), nprocs=xm.xrt_world_size(), start_method='fork')
else:
    gan.train(train_dataset,val_dataset,diffPairAugmentRate)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.plot(gan.gen_stats.losses, 'r', label='Generator Loss')
plt.plot(gan.desc_stats.losses, 'b', label='Descriminator Loss')
plt.plot(gan.val_gen_stats.losses, 'magenta', label='Generator Loss')
plt.plot(gan.val_desc_stats.losses, 'cyan', label='Descriminator Loss')
plt.legend()
plt.show()

# Generation for samples by best model

In [ ]:
ckpt_path = 'current.ckpt'
ckpt= load_checkpoint(ckpt_path)
print(ckpt.keys())
gan = CycleGAN(3, 3, 50, device)
gan.gen_s2p.load_state_dict(ckpt['gen_s2p'])
gan.gen_p2s.load_state_dict(ckpt['gen_p2s'])
gan.desc_s.load_state_dict(ckpt['desc_s'])
gan.desc_p.load_state_dict(ckpt['desc_p'])
gan.adam_gen.load_state_dict(ckpt['optimizer_gen'])
gan.adam_desc.load_state_dict(ckpt['optimizer_desc'])

In [ ]:
img_val = DataLoader(val_dataset,
                        batch_size=1,
                        num_workers=os.cpu_count(),
                        worker_init_fn=seed_worker,
#                                     generator=g,
                        pin_memory=True) 
img_val = iter(img_val)
for i in range(5):
    photo_img, _ = next(img_val)
    pred_style = gan.gen_p2s(photo_img.to(device)).cpu().detach()
    photo_img = unnorm(photo_img).permute(0,2, 3,1)
    pred_style = unnorm(pred_style).permute(0,2, 3,1)
    tmp = torch.cat((photo_img, pred_style), 0)
    if i == 0:
        forcheck_imgs = tmp
    else:
        forcheck_imgs = torch.cat((forcheck_imgs,tmp),0)
show_imgs(forcheck_imgs,5,2)

# Generation for submitting

In [ ]:
class MyDataset4Out(torch.utils.data.Dataset):

    def __init__(self, photo_dir, imageSize=(256,256), transform=None, normalize=True):
        if normalize:
            self.transform = transforms.Compose([
                transforms.Resize(imageSize), 
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(imageSize), 
                transforms.ToTensor(), 
            ])
        # Dataのパスリスト
        self.photo_paths = [str(p) for p in Path(photo_dir).glob("*.jpg")]

        self.data_num = len(self.photo_paths) # ここが__len__の返り値になる

    def __getitem__(self, idx):
        p = self.photo_paths[idx]
        photo_img = PIL.Image.open(p)
        if self.transform:
            out_photo = self.transform(photo_img)            
        return out_photo
        
    def __len__(self):
        return self.data_num

In [ ]:
photo_dataset = MyDataset4Out(photo_dir)
photo_dl = DataLoader(photo_dataset,
                        batch_size=1,
                        num_workers=os.cpu_count(),
                        worker_init_fn=seed_worker,
#                                     generator=g,
                        pin_memory=True)
os.makedirs('../images',exist_ok=True)

In [ ]:
t = tqdm(photo_dl, leave=False, total=photo_dl.__len__())
for i, photo in enumerate(t):
    gan.gen_p2s.eval()
    with torch.no_grad():
        styled_photo = gan.gen_p2s(photo.to(device))
        styled_photo = styled_photo.cpu().detach()
    styled_photo = unnorm(styled_photo)
    img = styled_photo[0].permute(1, 2, 0).numpy()
    #nyumpyからpilにする
    img_pil = PIL.Image.fromarray((img*255).astype(np.uint8))
    img_pil.save("../images/" + str(i+1) + ".jpg")

In [ ]:
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))